# **Bitcoin price forecasting - Random Forest Regressor**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



# Dependencies, Libraries and Tools

In [1]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
MODEL_NAME = "RandomForestRegressor"
SLOW_OPERATION = False

In [2]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
#Install some useful dependencies
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from itertools import cycle

import plotly.express as px

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import gc

import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

# !pip install -U -q PyDrive # To use files that are stored in Google Drive directly (e.g., without downloading them from an external URL)
# !apt install openjdk-8-jdk-headless -qq
# import os
# os.environ["JAVA_HOME"] = JAVA_HOME

In [4]:
# Install Spark and related dependencies
!pip install pyspark

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml import Pipeline

# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPriceForecasting").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=5b75021d284cfdd70103654dfd59fe046e37217811d99d333d1a1f065538dcf2
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


# Link to Google Drive

In [5]:
# Define GDrive paths
GDRIVE_DIR = "/content/drive"

GDRIVE_DATASET_OUTPUT_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/output"

GDRIVE_DATASET_NAME = "bitcoin_blockchain_data_30min"
GDRIVE_DATASET_NAME_ENG = GDRIVE_DATASET_NAME + "_eng"

GDRIVE_DATASET_NAME_EXT_ENG  = "/" + GDRIVE_DATASET_NAME_ENG + ".parquet"

GDRIVE_DATASET_NAME_ENG = GDRIVE_DATASET_OUTPUT_DIR + GDRIVE_DATASET_NAME_EXT_ENG


In [6]:
# Point Colaboratory to our Google Drive
from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


In [7]:
# Load datasets into pyspark dataframe objects
df = spark.read.load(GDRIVE_DATASET_NAME_ENG,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

# Import my utilities

In [8]:
import sys
GDRIVE_UTILITIES_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/utilities"
sys.path.append(GDRIVE_UTILITIES_DIR)

import shutil
shutil.rmtree(GDRIVE_UTILITIES_DIR + '/__pycache__')

import utilities

import importlib
importlib.reload(utilities)

<module 'utilities' from '/content/drive/MyDrive/BDC/project/utilities/utilities.py'>

# Evaluation of a simple model

In [9]:
GDRIVE_FEATURES_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/features"

GDRIVE_ALL_FEATURES_NAME = "all_features"
GDRIVE_COR_MATRIX_FEATURES_NAME = "cm_features"

GDRIVE_ALL_FEATURES_NAME_EXT = "/" + GDRIVE_ALL_FEATURES_NAME + ".json"
GDRIVE_COR_MATRIX_FEATURES_NAME_EXT = "/" + GDRIVE_COR_MATRIX_FEATURES_NAME + ".json"

GDRIVE_ALL_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_ALL_FEATURES_NAME_EXT
GDRIVE_COR_MATRIX_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_COR_MATRIX_FEATURES_NAME_EXT

In [10]:
# Loading all features
with open(GDRIVE_ALL_FEATURES, "r") as f:
    all_features = json.load(f)
print(all_features)

# Loading correlation matrix features
with open(GDRIVE_COR_MATRIX_FEATURES, "r") as f:
    cm_features = json.load(f)
print(cm_features)

# Set the depended variable
TARGET_VAL = 'tomorrow-market-price'

# Set the features label
FEATURES_LABEL = "features"

['total-bitcoins', 'market-cap', 'trade-volume', 'blocks-size', 'avg-block-size', 'n-transactions-total', 'n-transactions-per-block', 'hash-rate', 'difficulty', 'miners-revenue', 'transaction-fees-usd', 'n-unique-addresses', 'n-transactions', 'estimated-transaction-volume-usd']
['market-cap', 'miners-revenue', 'estimated-transaction-volume-usd', 'n-transactions-total', 'blocks-size', 'total-bitcoins', 'difficulty']


In [11]:
# Valid performances with all the features
utilities.evaluate_simple_model(df, all_features, MODEL_NAME, FEATURES_LABEL, TARGET_VAL)

Output hidden; open in https://colab.research.google.com to view.

In [12]:
# Valid performances with the corr matrix features
utilities.evaluate_simple_model(df, cm_features, MODEL_NAME, FEATURES_LABEL, TARGET_VAL)

Output hidden; open in https://colab.research.google.com to view.

# Hyperparameter tuning w/ Time-series cross validation

In [13]:
# Split proportion list
PORTION_LIST = [0.6, 0.7, 0.8, 0.9]

In [14]:
# RandomForest params
# params = {
#     'numTrees' : [3, 5, 10, 20, 30],# Number of trees to train, >=1, default:20
#     'maxDepth' : [3, 5, 10] # Maximum depth of the tree, <=30, default:5
# }

params = {
    'numTrees' : [5, 10, 15, 20, 25], # Number of trees to train, >=1, default:20
    'maxDepth' : [2, 3, 5, 7, 10] # Maximum depth of the tree, <=30, default:5
}

In [15]:
results = utilities.autoTuning(df, cm_features, params, PORTION_LIST, MODEL_NAME, FEATURES_LABEL, TARGET_VAL)
results

,Model,Proportion,Parameters,RMSE,MAPE,MAE,Variance,R2,Adjusted_R2,Time,Predictions
0,RandomForestRegressor,0.9,"[5, 3]",2675.385825,0.115558,2124.027616,3.981561e+06,-0.200674,-0.20115,0.978521,"DataFrame[tomorrow-market-price: double, predi..."


In [17]:
# RandomForest params
params = {
    'numTrees' : [20],# Number of trees to train, >=1, default:20
    'maxDepth' : [5] # Maximum depth of the tree, <=30, default:5
}

In [16]:
## Cross Validation Parameter
# Multiple Splits Time Series Cross Validation
mul_cv = {'cv_type':'mulTs',
          'kSplits': 5}

# Blocked Time Series Cross Validation
blk_cv = {'cv_type':'blkTs',
          'kSplits': 10}

In [18]:
results_mul_cv, trained_models_mul_cv = utilities.tsCrossValidation(df, cm_features, params, mul_cv, MODEL_NAME, FEATURES_LABEL, TARGET_VAL)
results_mul_cv

,Model,CV_type,Splits,Train&Validation,Parameters,RMSE,MAPE,MAE,Variance,R2,Adjusted_R2,Time
0,RandomForestRegressor,mulTs,1,"(21030, 21029)","[20, 5]",6649.607148,0.655152,5100.783408,2.592872e+07,-1.407229,-1.407801,1.619616
1,RandomForestRegressor,mulTs,2,"(42059, 21029)","[20, 5]",1400.716967,0.254167,1169.066752,3.439470e+06,0.595570,0.595474,1.936271
2,RandomForestRegressor,mulTs,3,"(63088, 21029)","[20, 5]",1045.463166,0.082272,783.612117,1.439319e+06,0.521382,0.521268,2.534902
3,RandomForestRegressor,mulTs,4,"(84117, 21029)","[20, 5]",32483.944526,0.648357,29561.734094,8.747917e+08,-4.180445,-4.181677,3.924634
4,RandomForestRegressor,mulTs,5,"(105146, 21029)","[20, 5]",12072.353729,0.511253,10790.782621,1.362583e+08,-0.542952,-0.543319,3.439943


In [19]:
results_blk_cv, trained_models_blk_cv = utilities.tsCrossValidation(df, cm_features, params, blk_cv, MODEL_NAME, FEATURES_LABEL, TARGET_VAL)
results_blk_cv

,Model,CV_type,Splits,Train&Validation,Parameters,RMSE,MAPE,MAE,Variance,R2,Adjusted_R2,Time
0,RandomForestRegressor,blkTs,1,"(10093, 2524)","[20, 5]",44.010916,0.071302,41.752315,2.032388e+03,-2.847101,-2.854740,1.123187
1,RandomForestRegressor,blkTs,2,"(10093, 2524)","[20, 5]",803.758295,0.302199,641.042601,4.108504e+05,-1.739058,-1.744497,1.140574
2,RandomForestRegressor,blkTs,3,"(10093, 2524)","[20, 5]",2253.340643,0.206191,2025.463693,6.040670e+06,0.013140,0.011181,1.303653
3,RandomForestRegressor,blkTs,4,"(10093, 2524)","[20, 5]",183.148782,0.016530,101.763194,2.297081e+03,0.238736,0.237225,1.793714
4,RandomForestRegressor,blkTs,5,"(10093, 2524)","[20, 5]",2383.908869,0.198378,2183.670535,4.769525e+06,-5.211015,-5.223349,1.141433
5,RandomForestRegressor,blkTs,6,"(10093, 2524)","[20, 5]",1131.593646,0.146997,908.816111,9.415286e+05,-0.580692,-0.583831,1.111942
6,RandomForestRegressor,blkTs,7,"(10093, 2524)","[20, 5]",9509.051696,0.215488,6553.643200,4.180399e+07,-0.851902,-0.855579,1.081672
7,RandomForestRegressor,blkTs,8,"(10093, 2524)","[20, 5]",1428.906669,0.025239,1180.911272,7.616658e+06,0.689597,0.688981,1.130637
8,RandomForestRegressor,blkTs,9,"(10093, 2524)","[20, 5]",11180.909685,0.389882,10108.697421,1.020245e+08,-4.412115,-4.422862,1.099462
9,RandomForestRegressor,blkTs,10,"(10093, 2524)","[20, 5]",1693.295016,0.065268,1523.135697,2.368022e+06,-1.975024,-1.980931,1.642387


# Comparison table

In [20]:
# Define what model_info and evaluators in the Model Comparison Table
model_info = ['Model','CV_type','Parameters']
evaluator_lst = ['RMSE','MAPE','MAE','Variance','R2','Adjusted_R2','Time']

# The the Cross Validation results would like to compare
comparison_lst = [results_mul_cv, results_blk_cv]

In [21]:
# Show the Comparison Table
pd.concat([utilities.modelComparison(cv_result, model_info, evaluator_lst) for cv_result in comparison_lst])

,Model,CV_type,Parameters,RMSE,MAPE,MAE,Variance,R2,Adjusted_R2,Time
0,RandomForestRegressor,mulTs,"[20, 5]",10730.417107,0.430240,9481.195798,2.083715e+08,-1.002735,-1.003211,2.691073
0,RandomForestRegressor,blkTs,"[20, 5]",3061.192422,0.163747,2526.889604,1.659801e+07,-1.667543,-1.672840,1.256866


# Training the final model

In [22]:
model = utilities.train_final_model(df, cm_features, params, MODEL_NAME, FEATURES_LABEL, TARGET_VAL)

In [23]:
GDRIVE_MODELS_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/models"
GDRIVE_MODEL_NAME_EXT = GDRIVE_MODELS_DIR + "/" + MODEL_NAME

In [24]:
# Save the trained model
model.write().overwrite().save(GDRIVE_MODEL_NAME_EXT)